# Post processing and Tree Segmentation

In [25]:
from whitebox import WhiteboxTools
import whitebox_workflows
import laspy
import pdal
import json
import numpy as np
import sys
from datetime import datetime
import rasterio

wbt = WhiteboxTools()
wbt.set_working_dir("D:/Masterarbeit/preprocessing")
print("Working directory:", wbt.work_dir)

wbe = whitebox_workflows.WbEnvironment()

Working directory: D:\Masterarbeit\preprocessing


# Ground classification with Simple Morphological Filter (SMRF) [Pingel et al., 2013]

In [ ]:
json_pipeline = {
    "pipeline": [
        {
            "type": "readers.las",
            "filename": "D:/Masterarbeit/Wenns_Data/PC/WennsUTM_small.las"
        },
        {
            "type": "filters.outlier"
        },
        {
            "type": "filters.smrf",
            "returns": "last",
            "where": "!(Classification == 7)",
            "slope": 0.2,
            "window": 16,
            "threshold": 0.45,
            "scalar": 1.2
        },
        {
            "type": "filters.expression",
            "expression": "Classification == 2"
        },
        {
            "type": "writers.las",
            "filename": "D:/Masterarbeit/output_lidar/Wenns_small_ground_smrf.las"
        }
    ]
}

json_pipeline = json.dumps(json_pipeline)

pipeline = pdal.Pipeline(json_pipeline)
count = pipeline.execute()

# Ground classification with Cloth Simulation Filter (CSF) [Zhang et al., 2016]

In [32]:
json_pipeline = {
    "pipeline": [
        {
            "type": "readers.las",
            "filename": "D:/Masterarbeit/Wenns_Data/PC/WennsUTM_small.las"
        },
        {
            "type": "filters.outlier"
        },
        {
            "type": "filters.csf",
            "resolution": 0.2,
        },
        # Export all
        {
            "type": "writers.las",
            "filename": "D:/Masterarbeit/output_lidar/las_ground_nonground.las",
        },

        # Ground export
        {
            "type": "writers.las",
            "filename": "D:/Masterarbeit/output_lidar/ground_csf.las",
            "where": "Classification == 2"
        },

        # Non-ground export
        {
            "type": "writers.las",
            "filename": "D:/Masterarbeit/output_lidar/nonground_csf.las",
            "where": "Classification != 2"
        }
    ]
}

json_pipeline = json.dumps(json_pipeline)

pipeline = pdal.Pipeline(json_pipeline)
count = pipeline.execute()

# Create DTM, DSM and CHM

In [ ]:
las_ground_nonground = laspy.read("output_lidar/las_ground_nonground.las")

xmin, xmax = np.min(las_ground_nonground.x), np.max(las_ground_nonground.x)
ymin, ymax = np.min(las_ground_nonground.y), np.max(las_ground_nonground.y)

print(xmin, xmax, ymin, ymax)

629070.5 629095.22 5221524.13 5221547.3


In [5]:
print(np.unique(las_ground_nonground.classification))

[0 1 2 7]


## Create DEM (IDW)

In [ ]:
print("Interpolating DEM...")
wbt.lidar_idw_interpolation(
i=r"D:\Masterarbeit\output_lidar\las_ground_nonground.las",
output="raw_dem.tif",
parameter="elevation",
returns="all",
resolution=0.2,
weight=1.0,
radius=2.5,
exclude_cls='0,1,7'
)

## Create DSM (IDW)

In [ ]:
print("Interpolating DSM...")
wbt.lidar_idw_interpolation(
i=r"D:\Masterarbeit\output_lidar\las_ground_nonground.las",
output="raw_dsm.tif",
parameter="elevation",
returns="first",
resolution=0.2,
weight=1.0,
radius=2.5
)

## Create CHM

In [3]:
print("Creating CHM...")
wbt.subtract(
    input1="raw_dem.tif",
    input2="raw_dsm.tif",
    output="raw_chm.tif"
)

Creating CHM...
.\whitebox_tools.exe --run="Subtract" --wd="D:\Masterarbeit" --input1='raw_dem.tif' --input2='raw_dsm.tif' --output='raw_chm.tif' -v --compress_rasters=False

****************************
* Welcome to Subtract      *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading data...
Progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
Progress: 13%
Progress: 14%
Progress: 15%
Progress: 16%
Progress: 17%
Progress: 18%
Progress: 19%
Progress: 20%
Progress: 21%
Progress: 22%
Progress: 23%
Progress: 24%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 28%
Progress: 29%
Progress: 30%
Progress: 31%
Progress: 32%
Progress: 33%
Progress: 34%
Progress: 35%
Progress: 36%
Progress: 37%
Progress: 38%
Progress: 39%
Progress: 40%
Progress: 41%
Progress: 42%
Progress: 43%
Progress: 44%
Progress: 45%
Progress: 46%
Progress: 47%
Pr

0

# Define CRS for DTM,DSM and CHM

In [ ]:
with rasterio.open("raw_dem.tif", "r+") as dataset:
    dataset.crs = "EPSG:32632"
    print(dataset.crs)

with rasterio.open("raw_dsm.tif", "r+") as dataset:
    dataset.crs = "EPSG:32632"
    print(dataset.crs)

with rasterio.open("raw_chm.tif", "r+") as dataset:
    dataset.crs = "EPSG:32632"
    print(dataset.crs)


EPSG:32632
EPSG:32632
EPSG:32632


# Normalize Pointcloud

In [ ]:
wbt.normalize_lidar(
    i=r"D:\Masterarbeit\output_lidar\nonground_csf.las",
    output=r"D:\Masterarbeit\output_lidar\normalized_pc.las",
    dtm="raw_dem.tif"
)

In [24]:
las_normalized = laspy.read("output_lidar/normalized_pc.las")
np.min(las_normalized.z)

np.float64(-1.26)

# Tree Segmentation with PyCrown

In [4]:
import sys
from datetime import datetime
print(sys.executable)
from pycrown import PyCrown

c:\Users\lukas\miniforge3\envs\pycrown\python.exe


## Set input files
Specify the file locations for the CHM, DSM, DTM and the LiDAR point cloud.
The latter is only needed if the point cloud should be classified into individual trees.

In [5]:
F_CHM = 'raw_chm.tif'
F_DTM = 'raw_dem.tif'
F_DSM = 'raw_dsm.tif'
F_LAS = 'output_lidar/normalized_pc.las'

## Initialize an instance of PyCrown

In [9]:
import laspy
print(laspy.__file__)
print(laspy.__version__)

c:\Users\lukas\miniforge3\envs\pycrown\lib\site-packages\laspy\__init__.py
1.7.0


In [10]:
PC = PyCrown(F_CHM, F_DTM, F_DSM, F_LAS, outpath='result')

RESOLUTIOON : 0.2


AttributeError: module 'laspy' has no attribute 'read'

## Clip all input data to new bounding box.

In [ ]:
PC.clip_data_to_bbox((1802150, 1802408, 5467305, 5467480))

## Smooth CHM
A 5x5m block median filter is used (set circular=True to enable a disc-shaped window).

In [ ]:
PC.filter_chm(5, ws_in_pixels=True, circular=False)

## Tree Detection with local maximum filter

In [ ]:
PC.tree_detection(PC.chm, ws=5, hmin=16.)

## Clip trees to bounding box 
(no trees on image edge)
original extent: 1802140, 1802418, 5467295, 5467490    

In [ ]:
PC.clip_trees_to_bbox(bbox=(1802160, 1802400, 5467315, 5467470))

## Crown Delineation

In [ ]:
PC.crown_delineation(algorithm='dalponteCIRC_numba', th_tree=15.,
                     th_seed=0.7, th_crown=0.55, max_crown=10.)

Tree crowns delineation: 0.007s


## (Optional) Correct tree tops on steep terrain

In [ ]:
PC.correct_tree_tops()

Number of trees: 128
Tree tops corrected: 9
Tree tops corrected: 7.03125%
DSM correction: 5
COM correction: 4


## Calculate tree height and elevation

In [ ]:
PC.get_tree_height_elevation(loc='top')
PC.get_tree_height_elevation(loc='top_cor')

## Screen small trees

In [ ]:
PC.screen_small_trees(hmin=20., loc='top')

## Convert raster crowns to polygons

In [ ]:
PC.crowns_to_polys_raster()
PC.crowns_to_polys_smooth(store_las=True)

Converting LAS point cloud to shapely points
Converting raster crowns to shapely polygons
Attach LiDAR points to corresponding crowns
Create convex hull around first return points
Classifying point cloud


## Check that all geometries are valid

In [ ]:
PC.quality_control()

## Print out number of trees

In [ ]:
print(f"Number of trees detected: {len(PC.trees)}")

Number of trees detected: 115


## Export results

In [ ]:
PC.export_raster(PC.chm, PC.outpath / 'chm.tif', 'CHM')
PC.export_tree_locations(loc='top')
PC.export_tree_locations(loc='top_cor')
PC.export_tree_crowns(crowntype='crown_poly_raster')
PC.export_tree_crowns(crowntype='crown_poly_smooth')